In [1]:
# Author: Tianyu Qi, Bohao Li, Chang Liu
# Course: CIS 600 - Data Mining and Social Media
# Date: 12/02/2018

In [32]:
# Required packages
import json
import twitter # This has to be installed
import urllib.parse as urllib
import pandas as pd
import nltk
import numpy as np
import re
import string
import random
import time
import datetime
import pytz
from datetime import datetime, tzinfo
from wordcloud import WordCloud # This has to be installed
from math import pi
from IPython.display import Image
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [6]:
# Required Bokeh Packages
import bokeh
from bokeh import events
from bokeh.layouts import row, column, widgetbox, layout
from bokeh.models.widgets import Button, TextInput, Select, Div, DataTable, TableColumn, NumberFormatter, Panel, Tabs, Paragraph
from bokeh.models import HoverTool, ColumnDataSource, GMapOptions, CustomJS
from bokeh.plotting import show, figure, gmap
from bokeh.io import show, push_notebook, output_notebook, reset_output
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models.tiles import WMTSTileSource
from bokeh.document import Document
from bokeh.palettes import Category10, Spectral
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.transform import cumsum
from bokeh.core.properties import value

In [31]:
# Install and import required nltk corpus
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import *
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Qitianyu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Qitianyu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
# Global Parameters
PORT = 8888
CREDFILE = 'OAuth.json'
RATE_LIMIT = 25
TRAINSET = 'data/train_data.csv'
APP_WIDTH = 650
APP_HEIGHT = 800
emotionVectorReverse = {0: "sadness", 1: "neutral", 2: "surprise", 3: "happiness", 4: "relief", 5: "fun", 6: "anger", 7: "bordom"}

In [10]:
# ----------------------------------< HELPER FUNCTIONS AND PROCESSES START >----------------------------

In [11]:
def getKeys(filename):
    """Parse keys file and extract the keys.
    
    Keyword arguments:
    fileName -- dir/and/fileName/of/the/keys
    
    Return values:
    keys     -- List of keys
    """
    with open(filename,'r') as fd:
        keys = json.load(fd)
    return keys

In [12]:
def authorizeTwitter(keys):
    """Authorize Twitter API using keys passed in.
    
    Keyword arguments:
    keys -- dictionary containes Twitter API keys and tokens
    
    Return values:
    api  -- Twitter API instance
    """
    api = twitter.Api(consumer_key = keys["consumer_key"], 
                consumer_secret = keys["consumer_secret"], 
                access_token_key = keys["token"], 
                access_token_secret = keys["token_secret"],
                sleep_on_rate_limit=True)
    return api

In [13]:
# Helper Function - Process User Query
def processQuery(term, api):
    """Compile the raw query and get the query result.
    
    Keyword arguments:
    term   -- The term for querying
    api    -- Twitter API instance
    
    Return values:
    tweets -- Raw result of the query
    """
    raw = "q=" + term.replace(" ", "") + "%20%23travel%20-filter%3Aretweets"
    flag = 1
    i = 0
    tweets = list();
    while (flag == 1):
        # Get tweets from Twitter search API
        results = api.GetSearch(raw_query = raw, return_json=True)
        i += 1
        tweets.append(results)
        # Check if there are more tweets could be harvested
        if ('next_results' in results['search_metadata'].keys()):
            raw = results['search_metadata']['next_results']
            temp = raw[1:].split('&q=')
            raw = '&q=' + temp[1] + '&' + temp[0]
        else:
            flag = 0
        if (i == RATE_LIMIT):
            flag = 0
    return tweets

In [14]:
def extractData(results):
    """Extract data from raw results.
    
    Keyword arguments:
    results -- The term for querying
    
    Return values:
    dataset -- Dataframe contains text and timestamp field of the tweets.
    """
    labels = ['text', 'timestamp']
    records = list()
    for i in range(len(results)):
        for j in range(len(results[i]['statuses'])):
            if ('text' not in results[i]['statuses'][j].keys()):
                text = results[i]['statuses'][j]['full_text']
            else:
                text = results[i]['statuses'][j]['text']
            timestamp = results[i]['statuses'][j]['created_at']
            records.append([text, timestamp])
    dataset = pd.DataFrame.from_records(records, columns = labels)
    return dataset

In [18]:
def getQueryWordList(word):
    """Parse the original text to list of tokens.
    
    Keyword arguments:
    word      -- String, the original tweet.
    
    Return values:
    result    -- The result of parsing.
    resultDup -- The result of parsing which contains duplicates.
    """
    wordListDup = tt.tokenize(word)
    wordList = list(set(tt.tokenize(word)))
    resultDup = []
    result = []
    for word in wordList:
        word = word.strip(string.punctuation)
        word = wnl.lemmatize(word)
        word = word.lower()
        if special.search(word) == None and word not in stops and len(word) > 1:
            result.append(word)
    for word in wordListDup:
        word = word.strip(string.punctuation)
        word = wnl.lemmatize(word)
        word = word.lower()
        if special.search(word) == None and word not in stops and len(word) > 1:
            resultDup.append(word)
    return (result, resultDup)

In [64]:
def processTrainData():
    """Extract data from raw results.
    
    Keyword arguments:
    None
    
    Return values:
    conditionSet -- Dictionary contains probability of each sentiment in the training set.
    wordSet      -- Dictionary contains conditional probability of each word in the training set under each sentiment.
    emotions     -- Dictionary contains probability of each sentiment in the training set. 
    """
    rawSet = pd.read_csv(TRAINSET)
    stops = set(stopwords.words('english'))
    wnl = WordNetLemmatizer()
    tt = TweetTokenizer()
    emotionVector = {"empty": -1, "sadness": 0, "worry": 0, "neutral": 1, "surprise": 2, 
                     "love": 3, "happiness": 3, "relief": 4, "fun": 5, "enthusiasm": 5,
                     "hate": 6, "anger": 6, "boredom": 7}
    special = re.compile('[0-9,\,,\:,\/,\=,\&,\;,\%,\$,\@,\#,\%,\^,\*,\(,\),\{,\},\[,\],\|,\>,\<,\-,\!,\?,\.\'\"]')
    total = 0
    emotions = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    conditionSet = dict()
    wordSet = dict()
    trainList = dict()
    
    for index, row in rawSet.iterrows():
        # Rule out unuseful sentiments
        if row["sentiment"] == "empty" or row["sentiment"] == "worry": 
            continue
        total += 1
        emotions[emotionVector[row["sentiment"]]] += 1
        sentence = row["content"]
        wordList = list(set(tt.tokenize(sentence)))
        for word in wordList:
            word = word.strip(string.punctuation)
            word = wnl.lemmatize(word)
            word = word.lower()
            if special.search(word) == None and word not in stops and len(word) > 1:
                if word not in trainList:
                    trainList[word] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
                trainList[word][emotionVector[row["sentiment"]]] += 1
    trainList = sorted(trainList.items(),key = lambda x: (x[1][0] + x[1][1] + x[1][2] + x[1][3] + x[1][4] + x[1][5] + x[1][6] + x[1][7]), reverse = True)
    trainList = trainList[:10000]
    for word in trainList:
        conditionSet[word[0]] = word[1]
        wordSet[word[0]] = (word[1][0] + word[1][1] + word[1][2] + word[1][3] + word[1][4] + word[1][5] + word[1][6] + word[1][7]) / total
    
    for (key, value) in conditionSet.items():
        for i in range(8):
            value[i] = value[i] / total
    for i in range(8):
        emotions[i] = emotions[i] / total
    return (conditionSet, wordSet, emotions)

In [85]:
def classify(wordList, wordDup, conditionSet, wordSet, emotionSet):
    """Classify the sentiment result of the list of word passed in.
    
    Keyword arguments:
    wordList     -- A list of word requires classification.
    wordDup      -- A list of word contains duplicate.
    conditionSet -- A dictionary of word contains the count of its total appearance
    wordSet      -- A list of word used in training
    emotionSet   -- A list of sentiments
    
    Return values:
    pos          -- The result of sentiment analysis.
    res          -- The probability of each sentiment.
    actualDup    -- A list of word contains duplicates which appears in training word list.
    """
    result = []
    actual = []
    actualDup = []
    for word in wordList:
        if word in conditionSet.keys():
            actual.append(word)
    if len(actual) < 2:
        return (-1, None, None)
    for word in wordDup:
        if word in conditionSet.keys():
            actualDup.append(word)
    general = 1.0
    for word in actual:
        general *= wordSet[word]
    for i in range(8):
        condition = emotionSet[i]
        for word in actual:
            condition *= conditionSet[word][i]
        if condition / general > 1.0: 
            result.append(0.9 + condition / general % 0.1)
        else:
            result.append(condition / general)
    res = 0.0
    pos = -1
    for i in range(8):
        if result[i] > res:
            res = result[i]
            pos = i
    return (pos, result, actualDup)

In [17]:
# Declare helper workers
stops = set(stopwords.words('english'))
wnl = WordNetLemmatizer()
tt = TweetTokenizer()
special = re.compile('[0-9,\,,\:,\/,\=,\&,\;,\%,\$,\@,\#,\%,\^,\*,\(,\),\{,\},\[,\],\|,\>,\<,\-,\!,\?,\.\'\"]')

In [86]:
def getEmotions(tweetList, timeList):
    """For each tweet in the list of tweets, carry out a sentiment analysis.
    
    Keyword arguments:
    tweetList        -- A list of original tweets.
    timeList         -- A list of post time of tweets
    
    Return values:
    emotions         -- The result of sentiment analysis for each tweet.
    emotionsAll      -- A list of probabilities of all sentiments of each tweet.
    emotionVocab     -- A dictionary contains pairs of emotions and vocabularies belongs to those emotions.
    emotionTweets    -- A dictionary contains pairs of emotions and tweets belongs to those emotions
    emotionTimeStamp -- A dictionary contains pairs of emotions and timestamp belongs to those emotions
    """
    emotions = {"sadness": 0, "neutral": 0, "surprise": 0, "happiness": 0, "relief": 0, "fun": 0, "anger": 0, "bordom": 0}
    emotionVocab = {"sadness": [], "neutral": [], "surprise": [], "happiness": [], 
                    "relief": [], "fun": [], "anger": [], "bordom": []}
    emotionTweets = dict()
    emotionsAll = []
    emotionTimeStamp = []
    i = 0
    for word in tweetList:
        (words, wordsDup) = getQueryWordList(word)
        (emotionResult, emotionAllRes, emotionWords) = classify(words, wordsDup, conditionSet, wordSet, emotionSet)
        if emotionResult != -1:
            emotions[emotionVectorReverse[emotionResult]] += 1
            emotionsAll.append(emotionAllRes)
            emotionTweets[word] = emotionVectorReverse[emotionResult]
            emotionTimeStamp.append(timeList[i])
            for emotionWord in emotionWords:
                emotionVocab[emotionVectorReverse[emotionResult]].append(emotionWord)
        i += 1
    return (emotions, emotionsAll, emotionVocab, emotionTweets, emotionTimeStamp)

In [20]:
def readableTime(timestampRaw):
    """Get readable time in string out of twitter timestamp format.
    
    Keyword arguments:
    timestampRaw -- A list time data in twitter timestamp format.
    
    Return values:
    res          -- A list of readable time string 
    """
    res = []
    for timestamp in timestampRaw:
        res.append(time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(timestamp,'%a %b %d %H:%M:%S +0000 %Y')))
    return res

In [21]:
def printWordClouds(wordList, name):
    """Generate word cloud images with specified name.
    
    Keyword arguments:
    wordlist -- A list words with duplicates.
    name     -- The specified name
    
    Return values:
    None
    """
    wordcloud = WordCloud(background_color='white',
                           width=1000,
                           height=600, 
                           margin=0
                         ).generate(wordList)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.savefig("data/" + name + ".png")
    
def getWordClouds(emotionVocab):
    """Drive routine for printWordClouds.
    
    Keyword arguments:
    emotionVocab -- A dictionary contains pairs of emotions and vocabularies belongs to those emotions.
    
    Return values:
    possible     -- A list of string contains all sentiments which could be drawn a word cloud.
    """
    dt = time.time()
    timeArray = time.localtime(dt)
    otherStyleTime = time.strftime("%Y_%m_%d_%H_%M_%S", timeArray)
    possible = dict()
    allWords = ""
    for (key, value) in emotionVocab.items():
        if len(value) != 0:
            possible[key] = otherStyleTime + "_" + key
            tmp = ""
            for word in value:
                tmp += " "
                tmp += word
            printWordClouds(tmp, otherStyleTime + "_" + key)
            allWords += (tmp + " ")
    printWordClouds(allWords, otherStyleTime + "_" + "all")
    possible["all"] = (otherStyleTime + "_" + "all")
    return possible

In [22]:
def plotPieChart(dataSet):
    """Drive routine for printWordClouds.
    
    Keyword arguments:
    emotionVocab -- A dictionary contains pairs of emotions and vocabularies belongs to those emotions.
    
    Return values:
    possible     -- A list of string contains all sentiments which could be drawn a word cloud.
    """
    data = pd.Series(dataSet).reset_index(name='value').rename(columns={'index':'sentiment'})
    data['angle'] = data['value']/data['value'].sum() * 2*pi
    data['color'] = Spectral[len(dataSet)]
    data['percent'] = data['value'] / sum(dataSet.values()) * 100

    p = figure(plot_height=350, title="Sentiment Distribution", toolbar_location=None,
           tools="hover", tooltips="@sentiment: @percent{0.2f} %", x_range=(-0.5, 1.0))

    p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend='sentiment', source=data)

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
#     show(p)
    return p

In [90]:
def plotTweets(tweets, probabilities, condition, tweetsTime):
    """Plot tweet data table.
    
    Keyword arguments:
    tweets        -- A list of tweet text.
    condition     -- The condition input in bokeh application
    tweetsTime    -- A list of tweet post time
    probabilities -- Dictionary of each tweets and its all probabilities
    
    Return values:
    dt            -- A bokeh datatable object of tweets.
    """
    if condition != "all":
        tmp = dict()
        tmpTime = []
        tmpProb = []
        i = 0
        for (key, value) in tweets.items():
            if value == condition:
                tmp[key] = value
                tmpTime.append(tweetsTime[i])
                tmpProb.append(probabilities[i])
            i += 1
        probabilities = tmpProb
        tweetsTime = tmpTime
        tweets = tmp
    sadness = []
    neutral = []
    surprise = []
    happiness = []
    relief = []
    fun = []
    anger = []
    bordom = []
    for i in range(len(probabilities)):
        sadness.append('{:.1%}'.format(probabilities[i][0]))
        neutral.append('{:.1%}'.format(probabilities[i][1]))
        surprise.append('{:.1%}'.format(probabilities[i][2]))
        happiness.append('{:.1%}'.format(probabilities[i][3]))
        relief.append('{:.1%}'.format(probabilities[i][4]))
        fun.append('{:.1%}'.format(probabilities[i][5]))
        anger.append('{:.1%}'.format(probabilities[i][6]))
        bordom.append('{:.1%}'.format(probabilities[i][7]))
    dataRaw = {"Tweet": list(tweets.keys()), "Post Time": readableTime(tweetsTime), "Sadness": sadness, "Happiness": happiness, "Neutral": neutral}
    data = pd.DataFrame.from_records(dataRaw, columns=["Tweet", "Post Time", "Sadness", "Happiness", "Neutral"])
    cols = [TableColumn(field='Tweet', title='Tweet'), 
            TableColumn(field='Post Time', title='Post Time'),
            TableColumn(field='Sadness', title='Sadness'), 
            TableColumn(field='Happiness', title='Happiness'), 
            TableColumn(field='Neutral', title='Neutral')]
    dt = DataTable(source=ColumnDataSource(data), columns=cols, width=APP_WIDTH, height=250)
    return dt

In [24]:
def plotTagCloud(dataset):
    """Plot a word cloud for select sentiment.
    
    Keyword arguments:
    dataset -- A list of word.
    
    Return values:
    p       -- A bokeh object.
    """
    p = figure(plot_width=750, plot_height=500, toolbar_sticky=True)
    p.line([-6,6], [0,0], line_width=4)
    p.line([0,0], [-6,6], line_width=4)
    source = ColumnDataSource(data=dict(x=[-6, 6, 0, 0],
                                    y=[0, 0, 6, -6],
                                    names=['unpleasant', 'pleasant', 'positive', 'negtive']))
    p.scatter(x='x', y='y', size=8, source=source)
    labels = LabelSet(x='x', y='y', text='names', level='glyph',
              x_offset=2, y_offset=2, source=source, render_mode='canvas')
    p.add_layout(labels)
    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    return p

In [25]:
def plotBarChart(dataSet, dataSetTime):
    """Plot a bar chart for tweets.
    
    Keyword arguments:
    dataset     -- A list of tweets.
    dataSetTime -- A list of tweet posted time
    
    Return values:
    p           -- A bokeh bar chart object.
    """
    dts = set()
    for timestamp in dataSetTime:
        dtRaw = pd.Timestamp(timestamp)
        dtRaw = dtRaw.replace(hour=0, minute=0, second=0)
        dts.add(dtRaw)
    dts = list(sorted(dts))
    dtsFine = []
    emoList = []
    for dt in dts:
        current = str(dt.year) + "/" + str(dt.month) + "/" + str(dt.day)
        dtsFine.append(current)
    dataToDraw = {
        "days": dtsFine
    }
    emotion = list(dataSet.values())
    for i in range(len(dataSetTime)):
        dtRaw = pd.Timestamp(dataSetTime[i])
        for j in range(len(dts)):
            if dtRaw.month == dts[j].month and dtRaw.day == dts[j].day:
                if not(emotion[i] in dataToDraw):
                    dataToDraw[emotion[i]] = [0] * len(dts)
                dataToDraw[emotion[i]][j] += 1
                if not (emotion[i] in emoList):
                    emoList.append(emotion[i])
    colors = Spectral[len(emoList)]
    p = figure(x_range=dtsFine, plot_height=250, title="Trends of Sentiments",
           toolbar_location=None, tools="hover", tooltips="$name @days: @$name")

    p.vbar_stack(emoList, x='days', width=0.618, color=colors, source=dataToDraw,
                 legend=[value(x) for x in emoList])

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    
    return p

In [93]:
def plotTabs(dataset, probabilities, imageNames, condition, tweets, datasetTimeStamp):
    """Plot the right tab of the application.
    
    Keyword arguments:
    dataset          -- A list of sentiment analysis result.
    imageNames       -- A list of word cloud image names.
    condition        -- The condition input in bokeh application
    tweets           -- The harvested tweets.
    datasetTimeStamp -- The timestamp of posted twitter
    
    Return values:
    p                -- A bokeh panel object.
    """
    # Content for Panel 1
    most = ""
    maxi = 0
    for (key, value) in dataset.items():
        if value > maxi:
            most = key
            maxi = value
    header1 = Div(text='<div align="Left" style="display:block"> \
                            <h3>Sentiment Analysis Result</h3>\
                            <h3 style="color:grey">The result of sentiment analysis is displayed as below, with a pie chart displays the proportions of each sentiment</h3> \
                            <h3>People\'s sentiment of this spot</h3> \
                            <h3 style="display:block">Most people feel </h3> \
                            <h3 style="color: blue">' + most + '</h3> \
                            <br/>\
                        </div>', width=700)
    
    pieChart = row(plotPieChart(dataset), height = 600, width = APP_WIDTH)
    
    # Content for Panel 2
    header2 = Div(text='<div align="Left" style="display:block"> \
                            <h3>Harvested tweets and trend of sentiment</h3> \
                            <h3 style="color:grey">The table below shows each harvested tweet with its sentiment analysis result.</h3> \
                        </div>', width=700)
    header21 = Div(text='<div align="left" style="display:block"> \
                            <h3 style="color:grey">The chart below shows the trend of sentiments in the last 7 days.</h3>\
                        </div>', width=700)
    select = imageNames["all"]
    if condition in imageNames.keys():
        select = imageNames[condition]
    dataTable = row(plotTweets(tweets, probabilities, condition, datasetTimeStamp), height = 300, width = APP_WIDTH)
    lineChart = row(plotBarChart(tweets, datasetTimeStamp))
    
    # Content for Panel 3
    header3 = Div(text='<div align="Left" style="display:block"> \
                            <h3>Word cloud for selected sentiment</h3> \
                            <h3 style="color:grey">The image below is generated according to the frequency of words both contained in tweet and word list of Bayes classifier</h3>\
                            <h3 style="color:grey">The words are from tweets with sentiment filtered by conditions on the left, the default value is all sentiments</h3> \
                        </div>', width=700)
    imgUrl = "http://localhost:8888/files/data/" + select + ".png"
    div = Div(text="<div style='margin:-10px'> \
                        <img src=" + imgUrl + ">" + \
                   "</div>", width=1000)

    # Layout
    tab1 = Panel(child=column([header1,pieChart], height=APP_HEIGHT, width=APP_WIDTH), title='General Analyze Result')
    tab2 = Panel(child=column([header2, dataTable, header21, lineChart], height=APP_HEIGHT, width=APP_WIDTH), title='Tweets & Trends')
    tab3 = Panel(child=column([header3, div], height=APP_HEIGHT, width=APP_WIDTH), title='Word Cloud')
    tabs = Tabs(tabs=[tab1, tab2, tab3])
    return tabs

In [73]:
def modify_document(doc):
    """Handler function of bokeh application
    
    Keyword arguments:
    doc -- bokeh application HTML document.
    
    Return values:
    None
    """
    def update():
        """Handler of request processing

        Keyword arguments:
        None

        Return values:
        None
        """
        locName = GUI.children[0].children[1].value
        condition = GUI.children[0].children[4].value
        if locName=='':
            GUI.children[2] = Div(text='<br/> \
                                        <div align="center" style="display:block"> \
                                             <h2>Error: Check your input</h2> \
                                        </div>', width=APP_WIDTH)
            return
        
        GUI.children[2] = Div(text='<div align="center" style="display:block"> \
                                      <h3>Getting Tweets...</h3> \
                                      <br><br> \
                                      <iframe src="http://www.narration.co.il/wp-content/uploads/2018/09/loader.gif" \
                                          width="200" height="200" frameBorder="0"> \
                                      </iframe> \
                                    </div>', width=APP_WIDTH)
        time.sleep(1)
        feeds = processQuery(locName, tAPI)
        if len(feeds) < 1:
            GUI.children[2] = Div(text='<br/> \
                                        <div align="center" style="display:block"> \
                                             <h2>Error: No Results Found</h2> \
                                        </div>', width=APP_WIDTH)
            return
        dataRaw = extractData(feeds)
        data = dataRaw["text"]
        dataTimeStamp = dataRaw["timestamp"]
        
        GUI.children[2] = Div(text='<div align="center" style="display:block"> \
                                      <h3>Classifying...</h3> \
                                      <br><br> \
                                      <iframe src="http://www.narration.co.il/wp-content/uploads/2018/09/loader.gif" \
                                          width="200" height="200" frameBorder="0"> \
                                      </iframe> \
                                    </div>', width=APP_WIDTH)
        time.sleep(1)
        (emotionData, emotionProbAll, emotionVocab, emotionTweets, emotionTimeStamp) = getEmotions(data, dataTimeStamp)
        possibleImages = getWordClouds(emotionVocab)
        
        # Display Results
        GUI.children[2] = Div(text='<div align="center" style="display:block"> \
                                      <h3>Ploting...</h3> \
                                      <br><br> \
                                      <iframe src="http://www.narration.co.il/wp-content/uploads/2018/09/loader.gif" \
                                          width="200" height="200" frameBorder="0"> \
                                      </iframe> \
                                    </div>', width=APP_WIDTH)
        time.sleep(1)
        GUI.children[2] = plotRight(emotionData, emotionProbAll, possibleImages, condition, emotionTweets, emotionTimeStamp)
        # Reset Menu
        GUI.children[0] = plotInputComplex()
        
    def plotInputComplex():
        """Plot the inputComplex on the left

        Keyword arguments:
        None

        Return values:
        p -- bokeh row object
        """
        # Location Input
        inputLabel = Div(text='<h3>Location Input</h3>', height=20)
        spotName = TextInput(value="", title='',sizing_mode='scale_width')
        
        # Condition Input
        conditionLabel = Div(text='<h3>Condition Input</h3>', height=20)
        select = Select(title="Sentiment", value="all", options=["all", "sadness", "neutral", "surprise", "happiness", 
                                                                "relief", "fun", "anger", "bordom"])      
        submit = Button(label='Submit', button_type='primary')
        submit.on_click(update)
        conditionSubmit = Button(label='Add condition', button_type='primary')
        conditionSubmit.on_click(update)
        menu = widgetbox([inputLabel, spotName, submit, conditionLabel, select, conditionSubmit], width=200)
        return menu
    
    def plotRight(data, probabilities, imageName, condition, emotionTweets, dataTimeStamp):
        """Plot the right panels

        Keyword arguments:

        Return values:
        None
        """
        if data == None and imageName == None and condition == None and emotionTweets == None and dataTimeStamp == None:
            intro = Div(text='<div align="center" style="display:block"> \
                                    <h2>Welcome to TravelTwitter!</h2> \
                                    <h3>Author: Tianyu Qi, Chang Liu, Bohao Li</h3> \
                              </div>', width=700)
            p = column([intro])
        else:
            p = plotTabs(data, probabilities, imageName, condition, emotionTweets, dataTimeStamp)
        return p
    seperator = Div(text='', sizing_mode='scale_height', width=75)
    GUI = row([plotInputComplex(), seperator, plotRight(None, None, None, None, None, None)], width=900, height = APP_HEIGHT)
    doc.add_root(GUI)
    
applicationHandler = FunctionHandler(modify_document)
app = Application(applicationHandler)
doc = app.create_document()

In [28]:
# ----------------------------------< HELPER FUNCTIONS AND PROCESSES END >----------------------------

In [29]:
# ----------------------------------< MAIN PROGRAM STARTS >----------------------------

In [33]:
# Initialize APIs
keys = getKeys(CREDFILE)
tAPI = authorizeTwitter(keys)

In [34]:
# Train the Bayes classifier, and declare helper variable
(conditionSet, wordSet, emotionSet) = processTrainData()

In [94]:
# Reset the output of bokeh application and output the application
reset_output()
output_notebook()
show(app, notebook_url="localhost:" + str(PORT), notebook_handle=True)

Loading BokehJS ...

In [681]:
# ----------------------------------< MAIN PROGRAM ENDS >----------------------------

In [682]:
# ----------------------------------< TEST & DEBUG STARTS >----------------------------

In [88]:
# For Testing and Debugging Purposes only
feeds = processQuery("las vegas", tAPI)
dataRaw = extractData(feeds)
data = dataRaw["text"]
dataTimeStamp = dataRaw["timestamp"]
(emotionData, emotionProbAll, emotionVocab, emotionTweets, emotionTimeStamp) = getEmotions(data, dataTimeStamp)
possibleImages = getWordClouds(emotionVocab)

In [92]:
# For Testing and Debugging Purposes only (Cont.)
show(plotTabs(emotionData, emotionProbAll, possibleImages, "all", emotionTweets, emotionTimeStamp))

In [71]:
# ----------------------------------< TEST & DEBUG ENDS >----------------------------